In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
titanic = pd.read_csv('data/titanic.csv')
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


#### 1. 전처리

In [15]:
# 전처리: 1) null 값 채우기
titanic['Age'].fillna(titanic['Age'].mean(),inplace=True)
titanic['Cabin'].fillna('N', inplace=True)
titanic['Embarked'].fillna('N', inplace=True)
print(titanic.isna().sum().sum())

0


In [16]:
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,N,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,N,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,N,S


In [21]:
# 2) collumns 삭제
titanic_df=titanic.drop(['Name','PassengerId','Ticket'], axis=1)
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,N,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,N,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,N,S


In [26]:
# 3) Cabin앞문자만 따오기
titanic_df['Cabin']=titanic_df['Cabin'].str[:1]

In [35]:
titanic_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,1,22.0,1,0,7.2500,N,S
1,1,1,0,38.0,1,0,71.2833,C,C
2,1,3,0,26.0,0,0,7.9250,N,S
3,1,1,0,35.0,1,0,53.1000,C,S
4,0,3,1,35.0,0,0,8.0500,N,S


In [68]:
# 4) 범주형 문자 --> 수치형으로 바꿈
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

lbe=LabelEncoder()
titanic_df['Sex']=lbe.fit_transform(titanic_df['Sex'])

In [69]:
oh=OneHotEncoder(sparse_output=False)
C_B=oh.fit_transform(titanic_df[['Cabin','Embarked']])

In [70]:
columns=np.hstack(['Cab_' + oh.categories_[0], 'Emb_'+ oh.categories_[1]])

In [71]:
titanic_df=pd.concat([titanic_df, pd.DataFrame(C_B, columns=columns)], axis=1)

In [73]:
titanic_df=titanic_df.drop(['Cabin','Embarked'], axis=1)

In [76]:
# 5) 종속변수와 독립변수 나누기
X_titanic=titanic_df.drop(['Survived'], axis=1)
y_titanic=titanic_df['Survived']

print(X_titanic.shape, y_titanic.shape)

(891, 32) (891,)


#### 2. 학습, 테스트 데이터 나누기

In [78]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(X_titanic, y_titanic, test_size=0.2, random_state=0)
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(712, 32) (179, 32) (712,) (179,)


#### 3. 학습, 예측, 정확도 검증

In [79]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

dt_clf=DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
pred=dt_clf.predict(X_test)
print(f'예측정확도: {accuracy_score(y_test,pred):.3f}')

예측정확도: 0.799


#### 4. 교차검증

In [82]:
from sklearn.model_selection import cross_val_score

scores=cross_val_score(dt_clf, X_titanic, y_titanic, cv=5)
for iter_count, accuracy in enumerate(scores):
    print(f'교차검증: {iter_count}, 정확도: {accuracy:.3f}')
print(f'평균 정확도: {np.mean(scores):.3f}')

교차검증: 0, 정확도: 0.743
교차검증: 1, 정확도: 0.764
교차검증: 2, 정확도: 0.831
교차검증: 3, 정확도: 0.770
교차검증: 4, 정확도: 0.826
평균 정확도: 0.787


In [87]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=7)
dt_clf = DecisionTreeClassifier(random_state=11)
acc = []
for train_index, test_index in kfold.split(X_titanic):
    X_train, X_test = X_titanic.iloc[train_index], X_titanic.iloc[test_index]
    y_train, y_test = y_titanic.iloc[train_index], y_titanic.iloc[test_index]   
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)   
    acc.append(round(accuracy_score(y_test, pred), 4))
print(acc)
print(np.mean(acc))

[0.7344, 0.75, 0.7795, 0.811, 0.8346, 0.7953, 0.8189]
0.7891


#### 5. 데이터 스케일링-표준화, 정규화

In [89]:
# StandardScale 표준화
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
result = scaler.fit_transform(titanic_df[['Age', 'SibSp', 'Parch', 'Fare']])
titanic_df[['Age', 'SibSp', 'Parch', 'Fare']] = result
titanic_df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cab_A,Cab_B,Cab_C,...,Cab_D,Cab_E,Cab_F,Cab_G,Cab_N,Cab_T,Emb_C,Emb_N,Emb_Q,Emb_S
0,0,3,1,-0.592481,0.432793,-0.473674,-0.502445,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,1,1,0,0.638789,0.432793,-0.473674,0.786845,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,3,0,-0.284663,-0.474545,-0.473674,-0.488854,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


#### 6. 재학습, 예측 등

In [91]:
y = titanic_df['Survived']
X = titanic_df.iloc[:, 1:]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=11)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(712, 32) (179, 32) (712,) (179,)


In [92]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier(random_state=0)
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, pred))

0.8268156424581006


In [93]:
# 한번 오차랭렬 해봄

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, pred)

array([[102,  16],
       [ 15,  46]], dtype=int64)